In [ ]:
#------------------------------------------------------------------------------
# 인구조사 데이터 이진 분류 ONLY CODE
#------------------------------------------------------------------------------

In [ ]:
#---------------------------------------
# 문제정의 
#---------------------------------------

In [ ]:
# 5만불 이상인지 아닌지 소득 예측

In [1]:
#1#
# 판다스 라이브러리
import pandas as pd

# 데이터 불러오기
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
# train = pd.read_csv('https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch2/train.csv')
# test = pd.read_csv('https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch2/test.csv')

In [2]:
#---------------------------------------
# EDA
#---------------------------------------
# 데이터 샘플
train.head(5)
train.tail(5)
train.sample(5)
# 데이터 크기
train.shape, test.shape
# 자료형 (타입)
train.info()
# 수치형 컬럼 통계값 확인(train)
train.describe()
# 수치형 컬럼 통계값 확인(test)
test.describe()
# 범주형 컬럼 통계값 확인 (train)
train.describe(include='O')
# 범주형 컬럼 통계값 확인 (test)
test.describe(include='object')
# 결측치 (train)
train.isnull().sum()
# 결측치 (test)
test.isnull().sum()
# label(target)별 개수 확인
train['income'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29304 entries, 0 to 29303
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              29304 non-null  int64  
 1   age             29292 non-null  float64
 2   workclass       27642 non-null  object 
 3   fnlwgt          29304 non-null  int64  
 4   education       29304 non-null  object 
 5   education.num   29304 non-null  int64  
 6   marital.status  29304 non-null  object 
 7   occupation      27636 non-null  object 
 8   relationship    29304 non-null  object 
 9   race            29304 non-null  object 
 10  sex             29304 non-null  object 
 11  capital.gain    29304 non-null  int64  
 12  capital.loss    29304 non-null  int64  
 13  hours.per.week  29291 non-null  float64
 14  native.country  28767 non-null  object 
 15  income          29304 non-null  object 
dtypes: float64(2), int64(5), object(9)
memory usage: 3.6+ MB


income
<=50K    22263
>50K      7041
Name: count, dtype: int64

In [ ]:
#---------------------------------------
# 데이터 전처리
#---------------------------------------

In [3]:
#---------------------
# 결측치 처리
#---------------------
# 결측치가 있는 데이터(행) 전체 삭제 및 확인 dropna() / 기본값 axis=0
print("처리전:", train.shape)
df = train.dropna()
print("처리후:", df.shape)

# 결측치가 있는 특정컬럼 데이터(행)삭제
df = train.dropna(subset=['native.country','workclass'])
df.isnull().sum()

# 결측치가 있는 컬럼 삭제 dropna(axis=1)
print("처리전:", train.shape)
df = train.dropna(axis=1)
print("처리후:", df.shape)

# 결측치가 많은 특정 컬럼 삭제 drop
print("처리전:", train.shape)
df = train.drop(['native.country','workclass'], axis=1)
print("처리후:", df.shape)

# 결측치채우기[범주형]  - 최빈값
m = train['workclass'].mode()[0]
train['workclass'] = train['workclass'].fillna(m)
m = train['native.country'].mode()[0]
train['native.country'] = train['native.country'].fillna(m)
train.isnull().sum()

# 결측치채우기[범주형]  - 임의값('X')
train['occupation'] = train['occupation'].fillna('X')
train.isnull().sum()

# 결측치 채우기[범주형] - test
test['workclass'] = test['workclass'].fillna(train['workclass'].mode()[0])
test['native.country']  = test['native.country'].fillna(train['native.country'].mode()[0])
test['occupation'] = test['occupation'].fillna("X")

# 결측치 채우기[수치형] - 평균값
value = int(train['age'].mean())
print("평균값:", value)
train['age'] = train['age'].fillna(value)
test['age'] = test['age'].fillna(value)

# 결측치 채우기[수치형] - 중앙값
value = int(train['hours.per.week'].median())
print("중앙값:", value)
train['hours.per.week'] = train['hours.per.week'].fillna(value)
test['hours.per.week'] = test['hours.per.week'].fillna(value)


#---------------------
# 이상치 처리
#---------------------
# age가 1이상인 데이터만 살림
print(train.shape)
train = train[train['age']>0]
print(train.shape)

#사분위수 이상치 범위 [예정]

#---------------------
# 인코딩[범주형 -> 수치형]
#---------------------
# 라벨(정답,y값) 분리
y_train = train.pop("income")
#----------
# 원핫인코딩
#----------
train_oh = pd.get_dummies(train)
test_oh = pd.get_dummies(test)
print(train.shape, test.shape, train_oh.shape, test_oh.shape)
#----------
# 불균형해결
#----------
print(train.shape, test.shape)
data = pd.concat([train, test], axis=0)
data_oh = pd.get_dummies(data)
train_oh = data_oh.iloc[:len(train)].copy()
test_oh = data_oh.iloc[len(train):].copy()
print(train_oh.shape, test_oh.shape)

#----------
# 레이블인코딩
#----------

cols = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex','native.country']
cols
from sklearn.preprocessing import LabelEncoder

for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

train.head()

#----------
# 레이블인코딩(개선 - test/train fit 맞지 않는경우
#----------
for col in cols:
    le = LabelEncoder()
    le.fit(pd.concat([train[col],test[col]] ,axis=0)) # 행합치기
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])
    

#----------
# y 인코딩
#----------
target = y_train.map({'<=50K':0, '>50K':1})
target.value_counts()
# or
target = y_train.replace('<=50K', 0).replace('>50K', 1)
target.value_counts()

# #---------------------
# # 스케일링(수치형데이터 범위 설정 - 이진분류에서는 x)
# #---------------------
# # 수치형 컬럼명
# cols = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week']
# # 데이터를 매번 새롭게 불러오기 위해 함수로 제작 함
# def get_data():
#     train_copy = train.copy()
#     test_copy = test.copy()
#     return train_copy, test_copy
# #----------
# #MINMAX
# #----------
# # 민-맥스 스케일링 MinMaxScaler (모든 값이 0과 1사이)
# train_copy, test_copy = get_data()

# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# display(train_copy[cols].head(2))
# train_copy[cols] = scaler.fit_transform(train_copy[cols])
# test_copy[cols] = scaler.transform(test_copy[cols])
# display(train_copy[cols].head(2))

# #----------
# #STANDARD
# #----------
# # 표준화 StandardScaler (Z-score 정규화, 평균이 0 표준편차가 1인 표준 정규분포로 변경)
# train_copy, test_copy = get_data()

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# display(train_copy[cols].head(2))
# train_copy[cols] = scaler.fit_transform(train_copy[cols])
# test_copy[cols] = scaler.transform(test_copy[cols])
# display(train_copy[cols].head(2))
# #----------
# #ROBUST
# #----------
# # 로버스트 스케일링 : 중앙값과 사분위 값 활용, 이상치 영향 최소화 장점
# from sklearn.preprocessing import RobustScaler
# scaler = RobustScaler()
# display(train[cols].head(2))
# train[cols] = scaler.fit_transform(train[cols])
# test[cols] = scaler.transform(test[cols])
# display(train[cols].head(2))



처리전: (29304, 16)
처리후: (27096, 16)
처리전: (29304, 16)
처리후: (29304, 11)
처리전: (29304, 16)
처리후: (29304, 14)
평균값: 38
중앙값: 40
(29304, 16)
(29301, 16)
(29301, 15) (3257, 15) (29301, 107) (3257, 103)
(29301, 15) (3257, 15)
(29301, 107) (3257, 107)


income
0    22262
1     7039
Name: count, dtype: int64

In [6]:
cols = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week']
def get_data():
    train_copy = train.copy()
    test_copy = test.copy()
    return train_copy, test_copy
#----------
#STANDARD
#---------
train_copy, test_copy = get_data()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
display(train_copy[cols].head(2))
train_copy[cols] = scaler.fit_transform(train_copy[cols])
test_copy[cols] = scaler.transform(test_copy[cols])
display(train_copy[cols].head(2))
# #----------
# #ROBUST
# #----------
# # 로버스트 스케일링 : 중앙값과 사분위 값 활용, 이상치 영향 최소화 장점
# from sklearn.preprocessing import RobustScaler
# scaler = RobustScaler()
# display(train[cols].head(2))
# train[cols] = scaler.fit_transform(train[cols])
# test[cols] = scaler.transform(test[cols])
# display(train[cols].head(2))

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,34.0,177331,10,4386,0,40.0
1,58.0,290661,9,0,0,40.0


,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,-0.335121,-0.117705,-0.031462,0.440247,-0.216056,-0.035121
1,1.428590,0.956277,-0.420430,-0.146298,-0.216056,-0.035121


In [ ]:
#---------------------------------------
# 검증 데이터 분할
#---------------------------------------

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train,
                                                  y_train,
                                                  test_size=0.2,
                                                  random_state=0)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((23440, 15), (5861, 15), (23440,), (5861,))

In [ ]:
#---------------------------------------
# 모델 학습
#---------------------------------------

In [4]:
# # 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
pred=rf.predict_proba(X_val)  
pred[:10]

# LightGBM
import lightgbm as lgb
lgbmc = lgb.LGBMClassifier(random_state=0, verbose=-1)
lgbmc.fit(X_train, y_train)
pred = lgbmc.predict_proba(X_val)

In [ ]:
#---------------------------------------
# 성능 평가
#---------------------------------------

In [5]:
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, pred[:,1])
print('roc_auc:', roc_auc)

from sklearn.metrics import accuracy_score
pred = rf.predict(X_val)
accuracy = accuracy_score(y_val, pred)
print('accuracy_score:', accuracy)

from sklearn.metrics import f1_score
f1 = f1_score(y_val, pred, pos_label='>50K')
print('f1_score:', f1)

roc_auc: 0.9277697320193289
accuracy_score: 0.8698174372973895
f1_score: 0.6936973103171417


In [ ]:
#---------------------------------------
# 파일로 저장
#---------------------------------------

In [8]:
pred = lgbmc.predict_proba(test)

lgbmc.classes_

submit = pd.DataFrame({'pred':pred[:,1]})
submit.to_csv("result.csv", index=False)

pd.read_csv("result.csv")

,pred
0,0.072682
1,0.023794
2,0.027383
3,0.826810
4,0.026526
...,...
3252,0.007832
3253,0.338767
3254,0.063109
3255,0.011152
